In [8]:
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input

In [9]:
from image_slicer import slice
from glob import glob
import os
import numpy as np
import pandas as pd

In [10]:
def prepare_image(path):
    # load an image from file
    image = load_img(path) # 224, 224

    # convert the image pixels to a numpy array
    image = img_to_array(image)

    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

    # prepare the image for the VGG model
    image = preprocess_input(image)
    
    return image

In [11]:
# load model
model = VGG16(input_shape=(100,100,3), include_top=False)

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

In [13]:
# out = GlobalAveragePooling2D()(model.output)
# model = Model(inputs=model.input, outputs=out)
# model.summary()

In [27]:
def feature_convergence(folder_path, model):
    data = []
    for i in range(len(os.listdir(folder_path))):
        image = folder_path+f'{i:03d}.png'
        slice(image, 81)
        img = []
        for j in range(1, 10):
            for k in range(1, 10):
                image= prepare_image(folder_path+f'{i:03d}_{j:02d}_{k:02d}.png')
                output = model.predict(image)
                output = output.flatten()
                img.append(np.mean(output)) # +np.std(output)
        for file in glob(folder_path+f'{i:03d}_*.png'):
            os.remove(file)
        data.append(img)
    return data
                

In [49]:
train_path = 'bottle/train/good/'
test_bs = 'bottle/test/broken_small/'
test_bl = 'bottle/test/broken_large/'
test_g = 'bottle/test/good/'
test_con = 'bottle/test/contamination/'

# test small broken

In [66]:
test_bs_data = feature_convergence(test_bs, model)
test_bs_data = pd.DataFrame(test_bs_data)

In [67]:
test_bs_data = pd.DataFrame(test_bs_data)

# broken large

In [44]:
test_bl_data = feature_convergence(test_bl, model)

In [46]:
test_bl_data = pd.DataFrame(test_bl_data)

In [51]:
# bl = []
# for i in range(len(test_bl_data)):
#     num = np.sum((train_df.mean()-test_bl_data.iloc[i])**2)
#     bl.append(num)
# bl

# test contamination

In [50]:
test_con_data = feature_convergence(test_con, model)
test_con_data = pd.DataFrame(test_con_data)

In [52]:
con = []
for i in range(len(test_con_data)):
    num = np.sum((train_df.mean()-test_con_data.iloc[i])**2)
    con.append(num)
con

[8.70897926890651,
 6.639637155684203,
 5.4362416529020905,
 3.586175123753173,
 5.541745800375883,
 7.842882237175408,
 4.606368530324701,
 9.426630267411271,
 7.275061881838697,
 6.343407411956487,
 14.799831189983971,
 2.7977860832412023,
 3.3184709375909662,
 3.4874228911002203,
 2.959565717190353,
 6.715153234220022,
 3.5307274573456544,
 5.730579191298196,
 4.751834266700814,
 2.4902590325588863,
 2.755529340002596]

# test good image

In [53]:
test_g_data = feature_convergence(test_g, model)
test_g_data = pd.DataFrame(test_g_data)

In [54]:
good = []
for i in range(len(test_g_data)):
    num = np.sum((train_df.mean()-test_g_data.iloc[i])**2)
    good.append(num)
good

[2.2624110381999305,
 2.026171384323102,
 4.155579696366053,
 2.999428417897528,
 1.588795302886484,
 1.8880411445436243,
 3.7639198499810593,
 3.4999415831221437,
 2.6471860856771396,
 3.7659832098505754,
 4.077082129775577,
 2.8108204880201715,
 3.5897448200753685,
 4.267787007526065,
 2.8403358675380117,
 3.8097490169842922,
 2.380374095897589,
 1.2740777617398387,
 3.876056124401651,
 4.1946134229112015]

# train good image

In [56]:
train_df = feature_convergence(train_path, model)
train_df = pd.DataFrame(train_df)

In [59]:
len(train_df)

209

# test the result

In [60]:
train_mse = []
for i in range(len(train_df)):
    num = np.sum((train_df.mean()-train_df.iloc[i])**2)
    train_mse.append(num)

In [62]:
np.min(train_mse), np.mean(train_mse), np.max(train_mse)

(1.002300012028511, 2.4857735318333067, 5.744649074135609)

In [81]:
train_mse

[2.5672482261179512,
 1.4885665936767722,
 2.5685065479776616,
 2.1318575561661337,
 1.9558573609548,
 2.192313294518267,
 3.2311272313426085,
 2.5554302329935528,
 1.7460863117056045,
 1.3150779876296417,
 2.4381736020287104,
 1.6492552961812026,
 2.5981915559538,
 3.367335199082297,
 1.450923479013376,
 2.2011557342458685,
 1.9083876750525937,
 1.5665861558199674,
 1.5718613540127837,
 2.164435098421674,
 2.136981455888738,
 2.2332554452435995,
 2.4078921724438276,
 2.3226547285198285,
 2.718797703311068,
 2.015772508724568,
 2.463057249337777,
 4.864094754139549,
 2.440541360509176,
 2.000463338112263,
 2.0918040966423797,
 2.5753544350993383,
 2.35933861391406,
 1.8353092485105327,
 2.887885522911284,
 2.1670317824466894,
 1.7735703521915407,
 2.1381160785006976,
 2.1155738652653735,
 3.9590502166675408,
 2.722146910773648,
 2.1555957870793847,
 2.3494436230187428,
 2.651074376713267,
 1.975607283534619,
 2.3128615059939848,
 5.744649074135609,
 1.9626202075365908,
 2.5599619355716

In [69]:
# train_df['data'] = 'train'
test_bs_data['data'] = 'broken_small'
test_bl_data['data'] = 'broken_large'
test_con_data['data'] = 'contamination'
test_g_data['data'] = 'good'

In [72]:
full_data = pd.concat([train_df, test_bs_data, test_bl_data, test_con_data, test_g_data])
full_data.to_csv('100x100fxdata.csv', index=False)

In [73]:
test_bs_mse = []
for i in range(len(test_bs_data)):
    num = np.sum((train_df.mean()-test_bs_data.iloc[i])**2)
    test_bs_mse.append(num)
np.min(test_bs_mse), np.mean(test_bs_mse), np.max(test_bs_mse)

(2.781633092135445, 4.30399617151289, 6.0191427467742)

In [80]:
test_bs_mse

[5.198811524040549,
 3.089243085109705,
 5.310222143842392,
 4.524341423150566,
 3.3531656524412816,
 4.158705613342506,
 3.998422188429854,
 2.781633092135445,
 4.498782104154644,
 3.671950048188515,
 5.198682565540642,
 4.257647224495963,
 6.0191427467742,
 4.065653946916187,
 3.295099249124982,
 5.320860567455062,
 4.965219112975054,
 5.378992002466773,
 3.265034264199912,
 3.4327955297636845,
 5.2082869426413545,
 3.6952247460943197]

In [74]:
test_bl_mse = []
for i in range(len(test_bl_data)):
    num = np.sum((train_df.mean()-test_bl_data.iloc[i])**2)
    test_bl_mse.append(num)
np.min(test_bl_mse), np.mean(test_bl_mse), np.max(test_bl_mse)

(2.3958971847371497, 16.328617562389404, 46.6766026153459)

In [79]:
test_bl_mse

[16.86579344075846,
 23.233734336665666,
 18.120357754369106,
 46.6766026153459,
 4.39253900600779,
 5.5250059228495845,
 14.951197750352147,
 36.2320431907434,
 19.550482256600425,
 11.681204703803681,
 20.734274302152404,
 2.3958971847371497,
 7.9199839366447575,
 14.095572697193719,
 6.966627140615768,
 11.69841565514853,
 11.856925704814524,
 14.979640364025721,
 24.301070183242988,
 14.394983101716388]

In [75]:
test_con_mse = []
for i in range(len(test_con_data)):
    num = np.sum((train_df.mean()-test_con_data.iloc[i])**2)
    test_con_mse.append(num)
np.min(test_con_mse), np.mean(test_con_mse), np.max(test_con_mse)

(1.783564545744268, 5.516882229078195, 15.355574560286271)

In [78]:
test_con_mse

[8.985517578415141,
 6.030769929829802,
 4.890872964370405,
 3.1164982826258836,
 6.529237497015559,
 7.833083011769635,
 4.519836426050578,
 9.240469029180613,
 7.777743767081272,
 6.600313038872734,
 15.355574560286271,
 2.5016922830032513,
 2.8585901710202113,
 3.2510602853920663,
 2.5596282195375166,
 6.248179718324462,
 2.985284728816897,
 5.988424444474304,
 4.324899196044618,
 1.783564545744268,
 2.473287132786625]

In [76]:
test_good_mse = []
for i in range(len(test_g_data)):
    num = np.sum((train_df.mean()-test_g_data.iloc[i])**2)
    test_good_mse.append(num)
np.min(test_good_mse), np.mean(test_good_mse), np.max(test_good_mse)

(1.331286452783834, 2.77835023438555, 3.9734758070236103)

In [77]:
test_good_mse

[1.988693807708115,
 1.7257951493141208,
 3.9734758070236103,
 2.4749551696398995,
 1.8445808989481949,
 1.634945924651387,
 3.217408378119193,
 3.419322117903049,
 2.193957410639437,
 3.2511582829738535,
 3.697655063521923,
 2.5433172375449296,
 3.2437686526949734,
 3.8683249015818086,
 2.365970702010338,
 3.6557245650862833,
 1.9688663420921733,
 1.331286452783834,
 3.5044764064047302,
 3.6633214170691537]